In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_6364/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_6364/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_9_7,0.965786,0.883308,0.820063,0.955190,0.014798,0.069274,0.042224,0.015536,0.056886,0.121646,0.954607,0.122557,74.426574,170.300641,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_45_5,0.969134,0.883024,0.785524,0.957539,0.013350,0.069442,0.050329,0.019656,0.054477,0.115541,0.959049,0.116406,74.632533,170.506600,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_43_2,0.968819,0.882843,0.783590,0.966558,0.013486,0.069549,0.050783,0.012303,0.054092,0.116128,0.958632,0.116997,74.612266,170.486334,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_43_5,0.968815,0.882325,0.783314,0.965304,0.013488,0.069857,0.050848,0.012764,0.054348,0.116136,0.958626,0.117006,74.611974,170.486042,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_45_3,0.969075,0.882180,0.785430,0.958796,0.013375,0.069943,0.050351,0.019074,0.054263,0.115651,0.958971,0.116517,74.628726,170.502794,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,model_5_0_4,0.395968,0.054872,-0.428242,0.364639,0.261244,0.561069,0.335151,0.244309,0.224004,0.511120,8.358206,0.514949,292.684602,713.949444,"Hidden Size=[36], regularizer=0.02, learning_r..."
857,model_6_0_0,0.027256,0.031260,-0.069966,0.014172,0.420712,0.575086,0.251078,0.314552,0.404583,0.648623,-0.195850,0.653482,51.731614,124.363484,"Hidden Size=[4, 2], regularizer=0.02, learning..."
858,model_4_0_0,-0.115294,-0.003995,-1.414557,-0.001904,0.482365,0.596015,0.566600,0.550088,0.431734,0.694525,-3.039174,0.699728,195.458108,477.269762,"Hidden Size=[24], regularizer=0.02, learning_r..."
859,model_2_0_0,-0.113485,-0.011781,-1.129293,-0.337506,0.481582,0.600637,0.499660,0.561718,0.460180,0.693961,-0.477296,0.699160,67.461356,163.335424,"Hidden Size=[8], regularizer=0.02, learning_ra..."
